<h1><center>TP2 - Ex.2</center></h1>
<p><center>Abril 1, 2024</center></p>


### Estruturas Criptográficas

PG53886, Ivo Miguel Alves Ribeiro

A95323, Henrique Ribeiro Fernandes


2. Uma das aplicações mais importantes do teorema chinês dos restos (CRT) em criptografia é a transformada NTT “Number Theoretic Transform”.  Esta transformada é uma componente importantes de “standards” PQC  como o Kyber e o Dilithium mas também de outros algoritmos submetidos ao concurso NIST PQC.  A transformação NTT tem várias opções e aquela que está apresentada no +Capítulo 4:  Problemas Difíceis  usa o CRT.
    Neste problema pretende-se uma implementação Sagemath  do NTT-CRT tal como é descrito nesse documento.

Estes são só os imports que precisamos para o nosso pograma

In [1]:
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.rings.finite_rings.finite_field_constructor import GF, FiniteField
from sage.rings.polynomial.polynomial_zmod_flint import Polynomial_zmod_flint
from sage.all import is_prime
from sage.arith.misc import CRT_basis

O primeiro passo é a escolha de um $N$ da forma $2^d$ e um primo $\,q\,$ que  verifique $$\,q \equiv 1 \bmod 2N\,$$

In [2]:
q = 40961
d = 12
N = pow(2,d) #4096

if ( q % (2* N)  == 1):
    print("A regra q ≡ 1 mod 2N é cumprida")
    while True:
        if is_prime(q):
            print(f"q tem o valor de:{q}")
            break
        q += 2*N
else:
    print("A regra q ≡ 1 mod 2N não é cumprida")
    
Zw = PolynomialRing(GF(q),name= "w" )
w = Zw.gen()


A regra q ≡ 1 mod 2N é cumprida
q tem o valor de:40961


#### Usando a estrategia apresntada no capitulo 4

1-Calculo dos monómios s e q

In [3]:
g = (w**N + 1)
xi = g.roots(multiplicities=False)[1]      
s = [xi * (xi**2)**i for i in range(N)]
q_i = [(w - s[i]) for i in range(N)]


2-Calculo do NTT

In [4]:
def NTT(xi, N1,f):
    if N1 == 1:
        return ([f[0]])

    f_plus, f_minus = split(f)
    bar_f_plus = NTT(xi**2, N1//2, f_plus)
    bar_f_minus = NTT(xi**2, N1//2, f_minus)

    s = xi

    bar_f = [0] * N1
 
    for i in range(N1//2):
        bar_f[i] = (bar_f_plus[i] + s * bar_f_minus[i])#.lift()
        bar_f[i + N1//2] = (bar_f_plus[i] - s * bar_f_minus[i])#.lift()
        s = s* (xi**2)
        
    return bar_f

def split (f):
    f_plus = sum(f[i] * w**(i/2) for i in range(f.degree()+1) if i % 2 == 0) + 0*w
    f_minus = sum(f[i] * w**((i-1)/2) for i in range(f.degree()+1) if i % 2 != 0) +0*w

    return f_plus,f_minus        

3-Operaçoes basicas do nosso anel

In [5]:
def subtracao_componentes(f, g):
    return [f[i] - g[i] for i in range(N)]

def soma_componentes(f, g):
    return [f[i] + g[i] for i in range(N)]

def multiplicacao_componentes(f, g):
    return [f[i] * g[i] for i in range(N)]

4-Reconstrução do NTT

In [6]:
def reconstruir_por_crt(bar_f):
    q_i = [(w - s[i]) for i in range(N)]
    mu_i = CRT_basis(q_i)
    return sum(bar_f[i] * mu_i[i] for i in range(N))

#### Teste do uso da tecnica NTT em uma funçao e de seguida a sua reconstruçao

In [7]:
f =  Zw.random_element(degree=N-1)

residuos_f = NTT(xi,N,f)

f1 = reconstruir_por_crt(residuos_f, )
print (f == f1)

True


#### Operaçaoes entre dois polinomios

In [8]:
############## teste 
f = Zw.random_element(degree=N-1)
g = Zw.random_element(degree=N-1)

residuos_f = NTT(xi,N,f)
residuos_g = NTT(xi,N,g)

op = input("Diga a operaçao que deseja realizar(+,*,-)")

# Realizando a operaçao resíduos
if (op == "*"):
    residuos_resultado = multiplicacao_componentes(residuos_f, residuos_g)
    # Reconstruindo o polinômio resultante usando CRT
    resultado_crt = reconstruir_por_crt(residuos_resultado)
elif (op == "+"):
    residuos_resultado = soma_componentes(residuos_f, residuos_g)
    print("Resultado correto? ")
    # Reconstruindo o polinômio resultante usando CRT
    resultado_crt = reconstruir_por_crt(residuos_resultado)
    print(f+g == resultado_crt)
elif (op == "-"):
    residuos_resultado = subtracao_componentes(residuos_f, residuos_g)
    print("Resultado correto? ")
    # Reconstruindo o polinômio resultante usando CRT
    resultado_crt = reconstruir_por_crt(residuos_resultado)
    print(f-g == resultado_crt)

print("Resultado da operação "+op+":", resultado_crt)


Resultado correto? 
True
Resultado da operação +: 29288*w^4095 + 40523*w^4094 + 39625*w^4093 + 16819*w^4092 + 1639*w^4091 + 2155*w^4090 + 26903*w^4089 + 3197*w^4088 + 29850*w^4087 + 39590*w^4086 + 1193*w^4085 + 15986*w^4084 + 5181*w^4083 + 23403*w^4082 + 11061*w^4081 + 14832*w^4080 + 10362*w^4079 + 30003*w^4078 + 10737*w^4077 + 10265*w^4076 + 25747*w^4075 + 37810*w^4074 + 32892*w^4073 + 2397*w^4072 + 16546*w^4071 + 16343*w^4070 + 18029*w^4069 + 26891*w^4068 + 9809*w^4067 + 5982*w^4066 + 13313*w^4065 + 27682*w^4064 + 3885*w^4063 + 17724*w^4062 + 40073*w^4061 + 13794*w^4060 + 8791*w^4059 + 38176*w^4058 + 11194*w^4057 + 31041*w^4056 + 34230*w^4055 + 40791*w^4054 + 19161*w^4053 + 30177*w^4052 + 1256*w^4051 + 935*w^4050 + 25952*w^4049 + 17359*w^4048 + 28590*w^4047 + 11211*w^4046 + 22830*w^4045 + 27417*w^4044 + 10166*w^4043 + 17007*w^4042 + 29101*w^4041 + 19261*w^4040 + 40053*w^4039 + 22229*w^4038 + 33188*w^4037 + 14765*w^4036 + 21195*w^4035 + 34770*w^4034 + 25153*w^4033 + 39453*w^4032 + 205